In [27]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from pandas import read_csv
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
#Now when creating your session pass this config to it.
sess = tf.Session(config=config)

ModuleNotFoundError: No module named 'sklearn'

In [23]:
# convert an array of values into a dataset matrix
def create_dataset(dataset, look_back=1):
	dataX, dataY = [], []
	for i in range(len(dataset)-look_back-1):
		a = dataset[i:(i+look_back), 0]
		dataX.append(a)
		dataY.append(dataset[i + look_back, 0])
	return numpy.array(dataX), numpy.array(dataY)
# fix random seed for reproducibility
numpy.random.seed(7)
# load the dataset
dataframe = read_csv('../data/zillow_2018.csv', usecols=[1], engine='python')
dataset = dataframe.values
dataset = dataset.astype('float32')

print (dataframe.head())

header_list = ['indicator_id','region_id','date','value','indicator','category','region_type','region']
data = pd.read_csv('../data/zillow_2018.csv', names=header_list)
df = pd.DataFrame(data)
print(df)

   region_id
0      99999
1      99999
2      99999
3      99999
4      99999
     indicator_id  region_id        date     value  \
0    indicator_id  region_id        date     value   
1            ZSFH      99999  2018-11-30  333069.0   
2            ZSFH      99999  2018-10-31  328005.0   
3            ZSFH      99999  2018-09-30  322212.0   
4            ZSFH      99999  2018-08-31  317946.0   
..            ...        ...         ...       ...   
491          ZSFH      99951  2018-05-31  333818.0   
492          ZSFH      99951  2018-04-30  332613.0   
493          ZSFH      99951  2018-03-31  330517.0   
494          ZSFH      99951  2018-02-28  327248.0   
495          ZSFH      99951  2018-01-31  323544.0   

                                    indicator     category  region_type  \
0                                   indicator     category  region_type   
1    ZHVI Single-Family Homes Time Series ($)  Home values          zip   
2    ZHVI Single-Family Homes Time Series ($)  H

In [26]:
# normalize the dataset
scaler = MinMaxScaler(feature_range=(0, 1))
dataset = scaler.fit_transform(dataset)

NameError: name 'MinMaxScaler' is not defined

In [ ]:
# split into train and test sets
train_size = int(len(dataset) * 0.67)
test_size = len(dataset) - train_size
train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]
# reshape into X=t and Y=t+1
look_back = 1
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)
# reshape input to be [samples, time steps, features]
trainX = numpy.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = numpy.reshape(testX, (testX.shape[0], 1, testX.shape[1]))
# create and fit the LSTM network
model = Sequential()
model.add(LSTM(4, input_shape=(1, look_back)))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(trainX, trainY, epochs=100, batch_size=1, verbose=2)
# make predictions
trainPredict = model.predict(trainX)
testPredict = model.predict(testX)
# invert predictions
trainPredict = scaler.inverse_transform(trainPredict)
trainY = scaler.inverse_transform([trainY])
testPredict = scaler.inverse_transform(testPredict)
testY = scaler.inverse_transform([testY])
# calculate root mean squared error
trainScore = math.sqrt(mean_squared_error(trainY[0], trainPredict[:,0]))
print('Train Score: %.2f RMSE' % (trainScore))
testScore = math.sqrt(mean_squared_error(testY[0], testPredict[:,0]))
print('Test Score: %.2f RMSE' % (testScore))
# shift train predictions for plotting
trainPredictPlot = numpy.empty_like(dataset)
trainPredictPlot[:, :] = numpy.nan
trainPredictPlot[look_back:len(trainPredict)+look_back, :] = trainPredict
# shift test predictions for plotting
testPredictPlot = numpy.empty_like(dataset)
testPredictPlot[:, :] = numpy.nan
testPredictPlot[len(trainPredict)+(look_back*2)+1:len(dataset)-1, :] = testPredict
# plot baseline and predictions
plt.plot(scaler.inverse_transform(dataset))
plt.plot(trainPredictPlot)
plt.plot(testPredictPlot)
plt.show()